In [1]:
import sys
import time
import pickle
import gzip
from random import randint
from scipy import misc
from scipy import special
import scipy.ndimage
from scipy.sparse import csc_matrix, issparse
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans,MiniBatchKMeans
import matplotlib.pyplot as plt
import json
import csv
import collections
import math
import sys

#setting path
DATA_PATH = 'data/mnist/'

IMAGES_TRAIN = 'data_training'
IMAGES_TEST = 'data_testing'

RANDOM_SEED = 42
N_CLASSES = 10
N_FEATURES = 28 * 28

#import data+label
data_training = DATA_PATH+IMAGES_TRAIN
data_testing = DATA_PATH+IMAGES_TEST
ft = gzip.open(data_training, 'rb')
TRAINING = pickle.load(ft)
ft.close()
ft = gzip.open(data_testing, 'rb')
TESTING = pickle.load(ft)
ft.close()

In [22]:
class NN:
    def __init__(self, neurons, stop_parameter, stop_function):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.stop_p = stop_parameter
        self.stop_f = stop_function
        self.best = 0.
        self.same = 0
        self.iteration = 0
        self.limit_epoch = 10
        
        #create random matrix for wheights
        np.random.seed(RANDOM_SEED)
        hidden_layer = np.random.rand(self.neurons, self.input_size + 1)/self.neurons
        output_layer = np.random.rand(self.output_size, self.neurons + 1) / self.output_size
        self.layers = [hidden_layer, output_layer]
        
    def train (self, training, testing):
        accu_train = 0. #[0., 0.]
        len_train = len(training[0])
        len_test = len(testing[0])
        self.start_time = dt.datetime.now()
        print('\n-- Training Session Start (%s) --' % (dt.datetime.now()))
        typeTrainingPrint = "Stop Function: " 
        if(self.stop_f==0):
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        else:
            typeTrainingPrint += "after "+str(self.limit_epoch)+" epochs"
        print(typeTrainingPrint)
        inputs = training[0][0:len_train]
        targets = np.zeros((len_train, 10))
        for i in range(len_train):
            targets[i, training[1][i]] = 1 #metto 1 nella posizione del valore della label
        
        while not self.is_stop_function(accu_train): #(accu_train[1]):
            self.iteration += 1
        
            for input_vector, target_vector in zip(inputs, targets):
                self.backprop(input_vector, target_vector)
        
        
            accu_test = self.accu(testing)
            accu_train = self.accu(training)
                
            #print (accu_train)
            
            if (self.iteration == 1 or self.iteration % 10 == 0):
                self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                
        if (self.iteration % 10 != 0):
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))

        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector):
        outputs = []
        for layer in self.layers:
            input_with_bias = np.append(input_vector, 1) 
            output = np.inner(layer, input_with_bias)
            output = special.expit(output)
            outputs.append(output)
            
            input_vector = output
        return outputs

    def backprop(self, input_vector, target):
        c = 10**(-4) + 10**(-1)/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector)
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        self.layers[-1] -= c*np.outer(output_deltas, np.append(hidden_outputs, 1))
        
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(self.layers[-1], self.neurons, 1).T, output_deltas)
        self.layers[0] -= c*np.outer(hidden_deltas, np.append(input_vector, 1))

    def predict(self, input_vector):
        return self.feed_forward(input_vector)[-1]

    def predict_one(self, input_vector):
        return np.argmax(self.feed_forward(input_vector)[-1])
    
    def accu(self, testing):
        res= np.zeros((10, 2))
        #se giusto aggiungo 1 ad entrambe le colonne di res (colonne = giusti, totali)
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k]) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        return np.round(total[0]/total[1]*100, 2)
    
    def is_stop_function(self, accuracy):
        if self.stop_f==0:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
        elif self.stop_f==1:
            if self.iteration>=self.limit_epoch:
                return True
            else:
                return False
            
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "+'Accuracy TRAIN: '+str(accu_train).zfill(4)+'%\t'+'Accuracy TEST: '+str(accu_test).zfill(4)+'%\t'
        print(message)
        
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
    def getWeight(self):
        return self.layers

In [23]:
nn = NN(neurons=100, stop_function=0, stop_parameter=0.01)
nn.train([TRAINING[0][0:12000],TRAINING[1][0:12000]], TESTING)
#[TRAINING[0][0:1000],TRAINING[1][0:1000]]


-- Training Session Start (2019-05-10 20:18:12.002720) --
Stop Function: improvements below 0.01%
Epoch 001 (9s)     Accuracy TRAIN: 89.84%	Accuracy TEST: 88.18%	
Epoch 010 (1m22s)  Accuracy TRAIN: 95.53%	Accuracy TEST: 93.23%	
Epoch 020 (2m42s)  Accuracy TRAIN: 96.78%	Accuracy TEST: 94.05%	
Epoch 030 (4m5s)   Accuracy TRAIN: 97.28%	Accuracy TEST: 94.48%	
Epoch 037 (5m3s)   Accuracy TRAIN: 97.56%	Accuracy TEST: 94.58%	

-- Training Session End (2019-05-10 20:23:15.420716) --


In [9]:
nn = NN(neurons=100, stop_function=1, stop_parameter=0.05)
nn.train(TRAINING, TESTING)


-- Training Session Start (2019-05-10 15:50:01.829679) --
Stop Function: after 10 epochs
Epoch 001 (30s)    Accuracy TRAIN: 92.84%	Accuracy TEST: 93.04%	
Epoch 002 (1m2s)   Accuracy TRAIN: 95.18%	Accuracy TEST: 95.03%	
Epoch 003 (1m32s)  Accuracy TRAIN: 96.05%	Accuracy TEST: 95.74%	
Epoch 004 (2m2s)   Accuracy TRAIN: 96.62%	Accuracy TEST: 96.13%	
Epoch 005 (2m32s)  Accuracy TRAIN: 96.95%	Accuracy TEST: 96.39%	
Epoch 006 (3m1s)   Accuracy TRAIN: 97.28%	Accuracy TEST: 96.62%	
Epoch 007 (3m31s)  Accuracy TRAIN: 97.55%	Accuracy TEST: 96.77%	
Epoch 008 (4m1s)   Accuracy TRAIN: 97.72%	Accuracy TEST: 96.91%	
Epoch 009 (4m31s)  Accuracy TRAIN: 97.86%	Accuracy TEST: 96.98%	
Epoch 010 (5m1s)   Accuracy TRAIN: 98.01%	Accuracy TEST: 97.03%	

-- Training Session End (2019-05-10 15:55:03.607212) --


In [24]:
def pruning_matrix(mat, percentage):
    threshold = (100-percentage)
    threshold /= 2
    perc_up, perc_down = 100 - threshold, threshold
    percentile_up = np.percentile(mat, perc_up)
    percentile_down = np.percentile(mat, perc_down)
    
    w_pruned = np.copy(mat)
    for i,row in enumerate(mat):
        for j,_ in enumerate(row):
            if mat[i,j] < percentile_down or mat[i,j] > percentile_up:
                w_pruned[i,j] = 0
    return w_pruned

def sparse_sub_dense(sparse,dense,mask):
    sparse.data -= dense.T[mask.T]

def delete_last_row(csc):
    i = csc.indptr[-1]
    indptr = csc.indptr[:-1]
    data = csc.data[:i]
    indices = csc.indices[:i]
    return csc_matrix((data,indices,indptr))    

In [25]:
class NN_Pr:
    def __init__(self, neurons, stop_parameter, stop_function, pruning, weights):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.stop_p = stop_parameter
        self.stop_f = stop_function
        self.best = 0.
        self.same = 0
        self.iteration = 0
        self.limit_epoch = 5
        self.pruning = pruning
        self.weights = weights
        
        #create random matrix for wheights
        np.random.seed(RANDOM_SEED)
        hidden_layer = csc_matrix(pruning_matrix(weights[0], pruning))
        output_layer = csc_matrix(pruning_matrix(weights[1], pruning))
        self.layers = [hidden_layer, output_layer]
        
        mask_hidden = hidden_layer.A != 0
        mask_output = output_layer.A != 0
        self.mask = [mask_hidden, mask_output]
        
    def train (self, training, testing):
        accu_train = 0.
        len_train = len(training[0])
        len_test = len(testing[0])
        self.start_time = dt.datetime.now()
        print('\n-- Training Session Start (%s) --' % (dt.datetime.now()))
        typeTrainingPrint = "Stop Function: " 
        if(self.stop_f==0):
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        else:
            typeTrainingPrint += "after "+str(self.limit_epoch)+" epochs"
        print(typeTrainingPrint)
        inputs = training[0][0:len_train]
        targets = np.zeros((len_train, 10))
        for i in range(len_train):
            targets[i, training[1][i]] = 1 #metto 1 nella posizione del valore della label
        
        while not self.is_stop_function(accu_train): #(accu_train[1]):
            self.iteration += 1
        
            for input_vector, target_vector in zip(inputs, targets):
                self.backprop(input_vector, target_vector)
        
        
            accu_test = self.accu(testing)
            accu_train = self.accu(training)
                
            #print (accu_train)
            
            if (self.iteration == 1 or self.iteration % 10 == 0):
                self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                
        if (self.iteration % 10 != 0):
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))

        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, input_vector):
        outputs = []
        for layer in self.layers:
            input_with_bias = np.append(input_vector, 1) 
            output = layer * input_with_bias #np.inner(layer, input_with_bias)
            output = special.expit(output)
            outputs.append(output)
            
            input_vector = output
        return outputs

    def backprop(self, input_vector, target):
        c = 10**(-4) + 10**(-1)/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(input_vector)
        
        output_deltas = outputs * (1 - outputs) * (outputs - target)
        #self.layers[-1] -= c*np.outer(output_deltas, np.append(hidden_outputs, 1))
        sparse_sub_dense(self.layers[-1], c*np.outer(output_deltas, np.append(hidden_outputs, 1)), self.mask[-1])
        
        #hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(np.delete(self.layers[-1], self.neurons, 1).T, output_deltas)
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * (delete_last_row(self.layers[-1]).T * output_deltas)
        sparse_sub_dense(self.layers[0], c*np.outer(hidden_deltas, np.append(input_vector, 1)), self.mask[0])

        #self.layers[0] -= c*np.outer(hidden_deltas, np.append(input_vector, 1))
        
    def predict(self, input_vector):
        return self.feed_forward(input_vector)[-1]

    def predict_one(self, input_vector):
        return np.argmax(self.feed_forward(input_vector)[-1])
    
    def accu(self, testing):
        res= np.zeros((10, 2))
        #se giusto aggiungo 1 ad entrambe le colonne di res (colonne = giusti, totali)
        for k in range(len(testing[1])):
            if self.predict_one(testing[0][k]) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        return np.round(total[0]/total[1]*100, 2)
    
    def is_stop_function(self, accuracy):
        if self.stop_f==0:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
        elif self.stop_f==1:
            if self.iteration>=self.limit_epoch:
                return True
            else:
                return False
            
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "+'Accuracy TRAIN: '+str(accu_train).zfill(4)+'%\t'+'Accuracy TEST: '+str(accu_test).zfill(4)+'%\t'
        print(message)
        
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
    def getWeight(self):
        return self.layers

In [26]:
w=nn.getWeight()
nnp = NN_Pr(neurons=100, stop_function=0, stop_parameter=0.01, pruning=75, weights=w)
nnp.train([TRAINING[0][0:12000],TRAINING[1][0:12000]], TESTING)


-- Training Session Start (2019-05-10 20:26:31.137213) --
Stop Function: improvements below 0.01%
Epoch 001 (16s)    Accuracy TRAIN: 89.42%	Accuracy TEST: 88.12%	
Epoch 010 (2m41s)  Accuracy TRAIN: 95.1%	Accuracy TEST: 93.12%	
Epoch 020 (5m22s)  Accuracy TRAIN: 96.34%	Accuracy TEST: 93.89%	
Epoch 030 (8m4s)   Accuracy TRAIN: 96.89%	Accuracy TEST: 94.12%	
Epoch 039 (10m39s) Accuracy TRAIN: 97.26%	Accuracy TEST: 94.48%	

-- Training Session End (2019-05-10 20:37:10.862413) --


In [21]:
w=nn.getWeight()
nnp1 = NN_Pr(neurons=100, stop_function=0, stop_parameter=0.01, pruning=60, weights=w)
nnp1.train([TRAINING[0][0:7000],TRAINING[1][0:7000]], [TESTING])


-- Training Session Start (2019-05-10 20:06:09.907970) --
Stop Function: improvements below 0.01%
Epoch 001 (16s)    Accuracy TRAIN: 87.27%	Accuracy TEST: 86.0%	
Epoch 002 (33s)    Accuracy TRAIN: 89.99%	Accuracy TEST: 88.0%	
Epoch 003 (49s)    Accuracy TRAIN: 90.97%	Accuracy TEST: 89.0%	
Epoch 004 (1m5s)   Accuracy TRAIN: 91.63%	Accuracy TEST: 89.6%	
Epoch 005 (1m21s)  Accuracy TRAIN: 92.24%	Accuracy TEST: 90.4%	
Epoch 006 (1m36s)  Accuracy TRAIN: 92.61%	Accuracy TEST: 91.0%	
Epoch 007 (1m52s)  Accuracy TRAIN: 93.09%	Accuracy TEST: 90.8%	
Epoch 008 (2m8s)   Accuracy TRAIN: 93.24%	Accuracy TEST: 90.8%	
Epoch 009 (2m23s)  Accuracy TRAIN: 93.43%	Accuracy TEST: 91.0%	
Epoch 010 (2m39s)  Accuracy TRAIN: 93.6%	Accuracy TEST: 91.2%	
Epoch 011 (2m56s)  Accuracy TRAIN: 93.73%	Accuracy TEST: 91.2%	
Epoch 012 (3m14s)  Accuracy TRAIN: 93.87%	Accuracy TEST: 91.4%	
Epoch 013 (3m29s)  Accuracy TRAIN: 93.96%	Accuracy TEST: 91.4%	
Epoch 014 (3m45s)  Accuracy TRAIN: 94.06%	Accuracy TEST: 91.2%	
Epoch 

In [2]:
import numpy as np
A = np.array([[1, 2, 3], [3, 4, 5]])

In [4]:
B = np.array([2,2])
np.outer(A, B)

array([[ 2,  2],
       [ 4,  4],
       [ 6,  6],
       [ 6,  6],
       [ 8,  8],
       [10, 10]])

In [1]:
import sys
import time
import pickle
import gzip
from random import randint
from scipy import misc
from scipy import special
import scipy.ndimage
from scipy.sparse import csc_matrix, issparse
import numpy as np
import datetime as dt
from sklearn.cluster import KMeans,MiniBatchKMeans
import matplotlib.pyplot as plt
import json
import csv
import collections
import math
import sys

#setting path
DATA_PATH = 'data/mnist/'

IMAGES_TRAIN = 'data_training'
IMAGES_TEST = 'data_testing'

RANDOM_SEED = 42
N_CLASSES = 10
N_FEATURES = 28 * 28

#import data+label
data_training = DATA_PATH+IMAGES_TRAIN
data_testing = DATA_PATH+IMAGES_TEST
ft = gzip.open(data_training, 'rb')
TRAINING = pickle.load(ft)
ft.close()
ft = gzip.open(data_testing, 'rb')
TESTING = pickle.load(ft)
ft.close()

In [60]:
class NN_b:
    def __init__(self, neurons, stop_parameter, stop_function):
        self.input_size = N_FEATURES
        self.output_size = N_CLASSES
        self.neurons = neurons
        self.stop_p = stop_parameter
        self.stop_f = stop_function
        self.best = 0.
        self.same = 0
        self.iteration = 0
        self.limit_epoch = 2
        
        #create random matrix for wheights
        np.random.seed(RANDOM_SEED)
        hidden_layer = np.random.rand(self.input_size, self.neurons)/self.neurons
        output_layer = np.random.rand(self.neurons, self.output_size)/self.output_size
        self.layers = [hidden_layer, output_layer]
        
    def train (self, training, testing):
        accu_train = 0.
        len_train = len(training[0])
        len_test = len(testing[0])
        self.start_time = dt.datetime.now()
        print('\n-- Training Session Start (%s) --' % (dt.datetime.now()))
        typeTrainingPrint = "Stop Function: " 
        if(self.stop_f==0):
            typeTrainingPrint += "improvements below "+str(self.stop_p)+"%"
        else:
            typeTrainingPrint += "after "+str(self.limit_epoch)+" epochs"
        print(typeTrainingPrint)
        inputs = training[0][0:len_train]
        targets = np.zeros((len_train, 10))
        for i in range(len_train):
            targets[i, training[1][i]] = 1 #metto 1 nella posizione del valore della label
        
        #onesForInput = np.ones((inputs.shape[0],1))
        #inputsWithB = np.hstack((inputs,onesForInput))
        
        while not self.is_stop_function(accu_train): #(accu_train[1]):
            self.iteration += 1
        
            #for input_vector, target_vector in zip(inputs, targets):
            self.backprop(inputs, targets)
        
        
            accu_test = self.accu(testing)
            accu_train = self.accu(training)
                
            #print (accu_train)
            
            if (self.iteration == 1 or self.iteration % 10 == 0):
                self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))
                
        if (self.iteration % 10 != 0):
            self.print_message_iter(self.iteration,accu_test,accu_train,self.ETAepoch(self.start_time))

        # Final message
        print('\n-- Training Session End (%s) --' % (dt.datetime.now()))

    def feed_forward(self, inputs):
        outputs = []
        for layer in self.layers:
            #da fare successivamente a monte    input_with_bias = np.append(input_vector, 1) 
            output = np.dot(inputs, layer) 
            output = special.expit(output)
            outputs.append(output)
            
            inputs = output
        return outputs #NumEsempi x neuroni; NumEsempi x classi

    def backprop(self, inputs, target):
        c = 10**(-4) + 10**(-1)/math.sqrt(self.iteration)  # Learning coefficient
        hidden_outputs, outputs = self.feed_forward(inputs)
        output_deltas = outputs * (1 - outputs) * (outputs - target) #NumEx x Output
        self.layers[-1] -= c*np.dot(hidden_outputs.T, output_deltas)
        print(self.layers[-1])
        hidden_deltas = hidden_outputs * (1 - hidden_outputs) * np.dot(output_deltas, self.layers[-1].T)
        self.layers[0] -= c*np.dot(inputs.T, hidden_deltas)

        
    def predict(self, inputs):
        return self.feed_forward(inputs)[-1]

   #def predict_one(self, inputs):
   #     return np.argmax(self.feed_forward(input_vector)[-1])
    
    def accu(self, testing):
        res= np.zeros((10, 2))
        #se giusto aggiungo 1 ad entrambe le colonne di res (colonne = giusti, totali)
        predicted = self.predict(testing[0])
        print(np.argmax(predicted[3]))
        print(testing[1][3])
        for k in range(len(testing[1])):
            if np.argmax(predicted[k]) == testing[1][k]:
            #if self.predict_one(testing[0][k]) == testing[1][k]:
                res[testing[1][k]] += 1
            else:
                res[testing[1][k]][1] += 1
        total = np.sum(res, axis=0)
        return np.round(total[0]/total[1]*100, 2)
    
    def is_stop_function(self, accuracy):
        if self.stop_f==0:
            if accuracy > self.best + self.stop_p or self.iteration == 0:
                self.best = accuracy
                return False
            else:
                return True
        elif self.stop_f==1:
            if self.iteration>=self.limit_epoch:
                return True
            else:
                return False
            
    def print_message_iter(self,iteration,accu_test,accu_train,eta):
        len_eta = len(eta)
        space_fill = 6 - len_eta
        eta = "("+eta+")"
        for _ in range(space_fill):
            eta += " "
        message = 'Epoch '+str(self.iteration).zfill(3) + " "+eta+" "+'Accuracy TRAIN: '+str(accu_train).zfill(4)+'%\t'+'Accuracy TEST: '+str(accu_test).zfill(4)+'%\t'
        print(message)
        
    def ETAepoch(self,start_time):
        diff = dt.datetime.now() - self.start_time
        eta = divmod(diff.days * 86400 + diff.seconds, 60)
        if eta[0] != 0:
            ret = str(eta[0])+"m"
        else:
            ret = ""
        ret += str(eta[1])+"s"
        return ret
    def getWeight(self):
        return self.layers

In [61]:
nnM = NN_b(neurons=15, stop_function=1, stop_parameter=0.01)
nnM.train([TRAINING[0][0:20],TRAINING[1][0:20]], TESTING)


-- Training Session Start (2019-05-12 15:26:34.382930) --
Stop Function: after 2 epochs
[[-0.18439378 -0.18564051 -0.17661767 -0.18679915 -0.18882382 -0.16840915
  -0.20797958 -0.24831564 -0.25170253 -0.22904723]
 [-0.25148125 -0.18396948 -0.2081127  -0.18782669 -0.13273655 -0.19941938
  -0.22011214 -0.19762326 -0.16539033 -0.16213895]
 [-0.2258114  -0.14004953 -0.16313939 -0.14045351 -0.16767623 -0.14573568
  -0.14871582 -0.19684796 -0.21122657 -0.16778733]
 [-0.17035995 -0.19540313 -0.13232841 -0.19175284 -0.1835928  -0.14315794
  -0.21065172 -0.20413562 -0.23792513 -0.19015002]
 [-0.23723348 -0.16080392 -0.19688715 -0.11339494 -0.13330384 -0.22095801
  -0.16245507 -0.24946878 -0.25075381 -0.22137037]
 [-0.15609976 -0.1001513  -0.21271652 -0.19437563 -0.20815475 -0.17880339
  -0.17656886 -0.23020242 -0.17580548 -0.19902194]
 [-0.16189641 -0.13137373 -0.18963032 -0.13286825 -0.15009277 -0.20278954
  -0.20030343 -0.16017144 -0.23640698 -0.22669414]
 [-0.20773377 -0.11563365 -0.1551628